In [ ]:
# Note: this training notebook has itself been derived from the nice example at
# https://github.com/warmspringwinds/pytorch-segmentation-detection/
# under recipes/pascal_voc/segmentation/resnet_34_8s_train.ipynb

# This script also uses the network model in
# warmspringwinds/pytorch-segmentation-detection/

%matplotlib inline
import sys, os
sys.path.insert(0, '../pytorch-segmentation-detection/vision/')
sys.path.append('../pytorch-segmentation-detection/')

# Choose GPU
os.environ["CUDA_VISIBLE_DEVICES"] = '1'


import torch
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

import numpy as np

import pytorch_segmentation_detection.models.fcn as fcns
import pytorch_segmentation_detection.models.resnet_dilated as resnet_dilated
from pytorch_segmentation_detection.transforms import (ComposeJoint,
                                                       RandomHorizontalFlipJoint,
                                                       RandomScaleJoint,
                                                       CropOrPad,
                                                       ResizeAspectRatioPreserve,
                                                       RandomCropJoint,
                                                       Split2D)

In [ ]:
sys.path.append("./dataset")
from labelfusion_masked import LabelFusionDataset
trainset = LabelFusionDataset()

# Note: in order to batch size larger than 1,
# appears that the tensor sizes of matches and non_matches
# will have to match across the batches
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=1, drop_last=True)


# ### massively important parameter ###
# set descriptor dimensionality here!!
# should maybe move this somewhere that stands out more
descriptor_dimensionality = 3

In [ ]:
%matplotlib notebook
from matplotlib import pyplot as plt

# Create the training plot
loss_current_iteration = 0
loss_history = []
match_loss_history = []
non_match_loss_history = []
consistency_loss_history = []
loss_iteration_number_history = []

f, (loss_axis, match_loss_axis, non_match_loss_axis, consistency_loss_axis) = plt.subplots(4, 1)

loss_axis.plot(loss_iteration_number_history, loss_history)
loss_axis.set_title('Total loss')
match_loss_axis.plot(loss_iteration_number_history, match_loss_history)
match_loss_axis.set_title('Training loss -- matches only')
non_match_loss_axis.plot(loss_iteration_number_history, non_match_loss_history)
non_match_loss_axis.set_title('Training loss -- non matches only')
consistency_loss_axis.plot(loss_iteration_number_history, consistency_loss_history)
consistency_loss_axis.set_title('Training loss -- consistency only')

plt.tight_layout()

In [ ]:
fcn = resnet_dilated.Resnet34_8s(num_classes=descriptor_dimensionality)
fcn.cuda()
fcn.train()

from pixelwise_contrastive_loss import PixelwiseContrastiveLoss
pixelwise_contrastive_loss = PixelwiseContrastiveLoss()

from semantic_consistency_loss import SemanticConsistencyLoss
semantic_consistency_loss = SemanticConsistencyLoss()

# note the lr specified here gets overwritten inside
# adjust_learning_rate
optimizer = optim.Adam(fcn.parameters(), lr=0.0001, weight_decay=0.0001)

load_previous = None
#load_previous = 'trained_models/debug_3d/dense_resnet_34_8s_04400.pth'
if load_previous is not None:
    print "loading", load_previous
    fcn.load_state_dict(torch.load(load_previous))
    optimizer.load_state_dict(torch.load(load_previous+".optim"))
    fcn.cuda()
    fcn.train()
    loss_current_iteration = int(load_previous.split('.pth')[0][-5:])
    print loss_current_iteration
    

def adjust_learning_rate(optimizer, iteration):
    """Pete: this also currently effectively stops training via an error
    since at max_iterations+1, it tries to raise a negative number to a fractional power"""
#     max_iteration = 10000.0
#     multiplier = (1.0 - (iteration / max_iteration)) ** (0.9)
#     lr = 0.00001 * multiplier
    if loss_current_iteration % 500 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr']*0.9
            
def save_net(iteration):
    folder_to_save = os.path.join(os.getcwd(),'trained_models/5drillscenes_maskedmatches_'+str(descriptor_dimensionality)+'d/')
    os.system("mkdir -p "+folder_to_save)
    net_to_save = 'dense_resnet_34_8s_'+str(iteration).zfill(5)+'.pth'
    net_to_save = os.path.join(folder_to_save, net_to_save)
    print "saving", net_to_save
    torch.save(fcn.state_dict(), net_to_save)
    torch.save(optimizer.state_dict(), net_to_save+".optim")

if loss_current_iteration == 0:
    save_net(0)

In [ ]:
import time

loss = match_loss = non_match_loss = consistency_loss = 0

for epoch in range(50):  # loop over the dataset multiple times

    for i, data in enumerate(trainloader, 0):
        loss_current_iteration += 1
        start_iter = time.time()

        # get the inputs
        data_type, img_a, img_b, matches_a, matches_b, non_matches_a, non_matches_b = data
        data_type = data_type[0]
        
        if len(matches_a[0]) == 0:
            print "didn't have any matches, continuing"
            continue
            
        img_a = Variable(img_a.cuda(), requires_grad=False)
        img_b = Variable(img_b.cuda(), requires_grad=False)
        
        W = 640
        H = 480
        N = 1
        
        if data_type == "matches":
            matches_a = Variable(matches_a.cuda().squeeze(0), requires_grad=False)
            matches_b = Variable(matches_b.cuda().squeeze(0), requires_grad=False)
            non_matches_a = Variable(non_matches_a.cuda().squeeze(0), requires_grad=False)
            non_matches_b = Variable(non_matches_b.cuda().squeeze(0), requires_grad=False)
            
        elif data_type == "masks":
            mask_a = Variable(matches_a.cuda().squeeze(0), requires_grad=False)
            mask_a = mask_a/torch.max(mask_a)
            mask_b = Variable(matches_b.cuda().squeeze(0), requires_grad=False)
            mask_b = mask_b/torch.max(mask_b)
            mask_a = mask_a.view(W*H,1).squeeze(1)
            mask_b = mask_b.view(W*H,1).squeeze(1)
            print mask_a.shape
            print torch.max(mask_a)
            print mask_b.shape
            print torch.max(mask_b)
        
            
        #print time.time() - start_iter, "loading inputs"
        #start = time.time()

        # zero the parameter gradients
        optimizer.zero_grad()
        adjust_learning_rate(optimizer, loss_current_iteration)
        
        #print time.time() - start, "putting on GPU and adjusting optimizer"
        #start = time.time()


        # forward + backward + optimize
        def process_net_output(image_pred):
            image_pred = image_pred.view(N,descriptor_dimensionality,W*H)
            image_pred = image_pred.permute(0, 2, 1)
            return image_pred


        image_a_pred = fcn(img_a)
        image_a_pred = process_net_output(image_a_pred)

        image_b_pred = fcn(img_b)
        image_b_pred = process_net_output(image_b_pred)
        
        #print time.time() - start, "forwarding"
        start = time.time()
        #print matches_a.shape

        # get loss
        if data_type == "matches":
            loss, match_loss, non_match_loss = pixelwise_contrastive_loss.get_loss(image_a_pred, image_b_pred, matches_a, 
                                            matches_b, non_matches_a, non_matches_b)
        elif data_type == "masks":
            loss = semantic_consistency_loss.get_loss(image_a_pred, image_b_pred,
                                                      mask_a, mask_b) 
            consistency_loss = loss
        
        #print time.time() - start, "getting loss"
        start = time.time()

        loss.backward()
        optimizer.step()
        
        #print time.time() - start, "backprop"
        start = time.time()

        loss_history.append(loss.data[0])
        match_loss_history.append(match_loss)
        non_match_loss_history.append(non_match_loss)
        loss_iteration_number_history.append(loss_current_iteration)
        consistency_loss_history.append(consistency_loss)
        
        if loss_current_iteration % 10 == 0:
            loss_axis.lines[0].set_xdata(loss_iteration_number_history)
            loss_axis.lines[0].set_ydata(loss_history)
            loss_axis.relim()
            loss_axis.autoscale_view()
            loss_axis.figure.canvas.draw()
            match_loss_axis.lines[0].set_xdata(loss_iteration_number_history)
            match_loss_axis.lines[0].set_ydata(match_loss_history)
            match_loss_axis.relim()
            match_loss_axis.autoscale_view()
            match_loss_axis.figure.canvas.draw()
            non_match_loss_axis.lines[0].set_xdata(loss_iteration_number_history)
            non_match_loss_axis.lines[0].set_ydata(non_match_loss_history)
            non_match_loss_axis.relim()
            non_match_loss_axis.autoscale_view()
            non_match_loss_axis.figure.canvas.draw()
            consistency_loss_axis.lines[0].set_xdata(loss_iteration_number_history)
            consistency_loss_axis.lines[0].set_ydata(consistency_loss_history)
            consistency_loss_axis.relim()
            consistency_loss_axis.autoscale_view()
            consistency_loss_axis.figure.canvas.draw()
        
        #print time.time() - start, "plotting"
        start = time.time()

        print time.time() - start_iter, "total seconds on gpu " + os.environ["CUDA_VISIBLE_DEVICES"]
        print (i, loss.data[0])
        if loss_current_iteration % 500 == 0:
            save_net(loss_current_iteration)

            #reset plots so they autozoom better
            loss_iteration_number_history = []
            loss_history = []
            match_loss_history = []
            non_match_loss_history = []
            consistency_loss_history = []
            
        if loss_current_iteration > 3500:
            raise ValueError("done")

print "Done!"



In [ ]:
raise ValueError("done")